<h1><b style="color: yellow;">Assignment 1</b></h1>


## <u> Imports

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from pathlib import Path

c:\Users\mishr\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = Ollama(model = 'mistral')

C:\Users\mishr\AppData\Local\Temp\ipykernel_23684\4158128036.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(model = 'mistral')


# <u> RAG

## Splitting text

In [3]:
with open('speech.txt','r',encoding='utf-8') as f:
    speech_text = f.read()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=40) # I kept the over lapping a little high for our task but i wanted to context to flow
texts = text_splitter.split_text(speech_text)
texts


['The real remedy is to destroy the belief in the sanctity of the shastras. How do you expect to',
 'of the shastras. How do you expect to succeed if you allow the shastras to continue to be',
 'held as sacred and infallible? You must take a stand against the scriptures. Either you must stop',
 'the scriptures. Either you must stop the practice of caste or you must stop believing',
 'in the shastras. You cannot have both. The problem of caste is not a problem of social reform. It',
 'is not a problem of social reform. It is a problem of overthrowing the authority of the',
 'shastras. So long as people believe in the sanctity of the shastras, they will never be able to get',
 'they will never be able to get rid of caste. The work of social reform is like the',
 'work of a gardener who is constantly pruning the leaves and branches of a tree without ever',
 'and branches of a tree without ever attacking the roots. The real enemy is the belief in the',
 'shastras.']

# Embedding fuction

In [4]:
embedding_function = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2'
)


C:\Users\mishr\AppData\Local\Temp\ipykernel_23684\3327000672.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(


# VectorDB (ChromaDB)

In [5]:
vector_store = Chroma(
    collection_name="Ambedkar",
    embedding_function=embedding_function,
    persist_directory="./chroma_langchain_db",
)
vector_store.add_texts(texts)

['861f41a0-962f-44ac-aba7-f5bd7d98998f',
 '587cc8ff-e7b7-4a7e-898d-f81bb54b321b',
 '3f4b783c-4a02-4693-92d4-460232386c21',
 '3542b33c-5730-48c7-9e9e-8ac2e2e6acc9',
 '6a10f943-1db4-41ca-976e-f4d38e679c34',
 '5d931228-f8f4-45ff-ba50-982e4e114d4e',
 '49abd8ff-1018-453b-9cab-cacc38096465',
 '799c6e03-2241-4c15-9381-117fe6c58b67',
 '702f9274-895d-49ca-935c-c19ed58d443d',
 '814a236f-ae98-4c49-bc2b-19aceadafe49',
 '7305c188-6cb5-43ae-a2e4-59b1a19a91f5']

# RetrievalQA 

In [6]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# Chains

In [7]:
prompt = ChatPromptTemplate.from_template("""
Use ONLY the context below to answer the question.

Context:
{context}

Question:
{question}

Answer:
""")


In [8]:
rag_chain = (
    {
        "question": lambda x: x["question"],
        "context": lambda x: retriever.invoke(x["question"]),
    }
    | prompt
    | llm
)

## Results

In [9]:
result = rag_chain.invoke({"question": "What is the remedy for caste system?"})
print(result)

 The provided context does not explicitly state a remedy for the caste system. However, it suggests that stopping the practice of caste or stopping the belief in it could potentially be a solution, as indicated in the second and third documents' page_content statements.


In [13]:
docs = retriever.invoke("What is the remedy for caste system?")
answer = rag_chain.invoke({"question": "What is the remedy for caste system?"})

print("ANSWER:\n", answer)


ANSWER:
  Based on the provided context, it suggests that stopping either the practice of caste or believing in the scriptures that uphold this system could potentially be a remedy for the caste system. However, the texts do not explicitly state a specific solution or remedy.
